<a href="https://colab.research.google.com/github/chandrakar-shubham/Python-Engineer-Assesment/blob/main/Python%20Engineer%20Assesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries to Import

In [118]:
import pandas as pd
import csv
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from zipfile import ZipFile
import logging

#Created function

1. download_file: Download any file with file name and url provided
2. get_zip_url : Extract zip file from xml file
3. create_csv : Creates CSV from XML File
4. extract_zip : function to extract zip file to a directory

In [140]:
def download_file(save_location,url,file_name):
  
  '''Download any file with file name and url provided ,
     local_filename consists of save_location = location to save file after download
     url = URL from which file will be downloaded
     file_name = file name provided by user'''
  
  
  
  local_filename = save_location + file_name
  #local_filename = '/content/drive/MyDrive/Python Engineer Assesment/'+ file_name
  
  
  # NOTE the stream=True parameter below
  with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
      for chunk in r.iter_content(chunk_size=1024): 
        # If you have chunk encoded response uncomment if
        # and set chunk_size parameter to None.
        #if chunk: 
        f.write(chunk)
  return (local_filename)

In [120]:
def get_zip_url(xml_file_path):
  
  '''Extract zip file from xml file'''
  
  
  empty_list=[]
  
  f = open(xml_file_path, 'r')
  
  xml_file = f.read()
  
  soup = BeautifulSoup(xml_file, 'lxml')
  
  
  for tag in soup.findAll("str"):
    #print(tag)
    
    if tag["name"] == 'download_link':
      empty_list.append(tag.text)
  return empty_list
  
  f.close()


In [121]:
def create_csv(xml_file, csv_path):
    """ Creates CSV from XML File
    Param(s):
        xml_file (str)  :   Path of XML file
        csv_path (str)  :   Path to write csv file
    Return(s):
        csv_file (str)  :   Absolute path of csv file
    """
    try:
        # Checking if the path exists or not
        if not os.path.exists(csv_path):
            # Creating the path
            logging.warning("Creating CSV file path")
            os.makedirs(csv_path)

        # Extracting the csv file name from xml file
        csv_fname = xml_file.split(os.sep)[-1].split(".")[0] + ".csv"

        # Creating csv file path
        csv_file = os.path.join(csv_path, csv_fname)

        logging.warning("Loading the xml file")
        # Creating xml file itertor
        xml_iter = ET.iterparse(xml_file, events=("start",))

        csv_columns = [
            "FinInstrmGnlAttrbts.Id",
            "FinInstrmGnlAttrbts.FullNm",
            "FinInstrmGnlAttrbts.ClssfctnTp",
            "FinInstrmGnlAttrbts.CmmdtyDerivInd",
            "FinInstrmGnlAttrbts.NtnlCcy",
            "Issr",
        ]

        # Creating empty dataframe with the required column names
        df = pd.DataFrame(columns=csv_columns)

        # List to store the extacted data
        extracted_data = []

        logging.warning("Parsing the xml file...")
        logging.warning("Extracting the required data from xml")
        # Traversing the xml data
        for event, element in xml_iter:

            # Checking for start of the tags
            if event == "start":

                # Checking for TermntdRcrd tag in which the required data is
                if "TermntdRcrd" in element.tag:

                    # Dictionary to store require data in single element
                    data = {}

                    # List of the required tags (FinInstrmGnlAttrbts, Issr)
                    reqd_elements = [
                        (elem.tag, elem)
                        for elem in element
                        if "FinInstrmGnlAttrbts" in elem.tag or "Issr" in elem.tag
                    ]

                    # Traversing through the required tags
                    for tag, elem in reqd_elements:

                        if "FinInstrmGnlAttrbts" in tag:

                            # Traversing through the child elements of
                            # FinInstrmGnlAttrbts element
                            for child in elem:

                                # Adding the extrcated data in the dictionary
                                if "Id" in child.tag:
                                    data[csv_columns[0]] = child.text
                                elif "FullNm" in child.tag:
                                    data[csv_columns[1]] = child.text
                                elif "ClssfctnTp" in child.tag:
                                    data[csv_columns[2]] = child.text
                                elif "CmmdtyDerivInd" in child.tag:
                                    data[csv_columns[3]] = child.text
                                elif "NtnlCcy" in child.tag:
                                    data[csv_columns[4]] = child.text

                        # Extracting Issr Tag value
                        else:
                            data[csv_columns[5]] = child.text

                    # Appending the single element extracted data in the list
                    extracted_data.append(data)

        logging.warning("All the required data extracted from xml file")

        # Appending the extracted data in the data frame
        df = df.append(extracted_data, ignore_index=True)

        logging.warning("Dropping empty rows")
        # Removes empty rows from the dataframe
        df.dropna(inplace=True)

        logging.warning("Creating the CSV file")
        # Creates csv file from the dataframe
        df.to_csv(csv_file, index=False)

        # returning the csv file path
        return csv_file

    except Exception as e:
        print(f"Error occurred while extracting - {str(e)}")


In [122]:
def extract_zip(url_zip,save_location):
  '''function to extract zip file to a directory'''


  # specifying the zip file name
  file_name = url_zip
  loc = save_location
  
  # opening the zip file in READ mode
  with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall(path=loc)
    print('Done!')

#Implementation of instructions

In [123]:
#url_path from where xml file to be downloaded


url_path='https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2021-01-17T00:00:00Z+TO+2021-01-19T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100'

In [124]:
#save_location for xml file after download


save_location = '/content/drive/MyDrive/Python Engineer Assesment/'

In [125]:
#xml_location after being downloaded

xml_loc = save_location + 'xml_file.xml'

In [126]:
#download and save xml_file to directory
download_file(save_location,url_path,'xml_file.xml')

'/content/drive/MyDrive/Python Engineer Assesment/xml_file.xml'

In [127]:
zip_url_list = get_zip_url(xml_loc)

In [128]:
zip_url_list

['http://firds.esma.europa.eu/firds/DLTINS_20210117_01of01.zip',
 'http://firds.esma.europa.eu/firds/DLTINS_20210119_01of02.zip',
 'http://firds.esma.europa.eu/firds/DLTINS_20210119_02of02.zip',
 'http://firds.esma.europa.eu/firds/DLTINS_20210118_01of01.zip']

In [129]:
#download zip file to a directory


for i in range(len(zip_url_list)):
  download_file(save_location,zip_url_list[i],'zipfile'+str(i))

In [130]:
#extract content of all zip files in a directory

for i in range(4):
  extract_zip(loc+'/zipfile'+str(i),save_location)

File Name                                             Modified             Size
DLTINS_20210117_01of01.xml                     2021-01-17 01:17:12    143278061
Extracting all the files now...
Done!
File Name                                             Modified             Size
DLTINS_20210119_01of02.xml                     2021-01-19 03:21:08    521464972
Extracting all the files now...
Done!
File Name                                             Modified             Size
DLTINS_20210119_02of02.xml                     2021-01-19 03:21:56    363082618
Extracting all the files now...
Done!
File Name                                             Modified             Size
DLTINS_20210118_01of01.xml                     2021-01-18 01:21:22      1439631
Extracting all the files now...
Done!


In [131]:
#input_extracted_xml consist list of xml files extracted from zip


input_extracted_xml=["/content/drive/MyDrive/Python Engineer Assesment/DLTINS_20210118_01of01.xml",
                     "/content/drive/MyDrive/Python Engineer Assesment/DLTINS_20210118_01of01.xml",
                     "/content/drive/MyDrive/Python Engineer Assesment/DLTINS_20210119_01of02.xml",
                     "/content/drive/MyDrive/Python Engineer Assesment/DLTINS_20210119_02of02.xml"]


#output_path_converted_csv is path where converted csv wil be saved



output_path_converted_csv="/content/drive/MyDrive/Python Engineer Assesment/"

In [132]:
#importing os function
import os

In [133]:
# for loop to covert all xml files extracted from zip


for link in input_extracted_xml:
  print(link)

  #calling create_csv function to convert xml to csv
  
  create_csv(link, output_path_converted_csv)


/content/drive/MyDrive/Python Engineer Assesment/DLTINS_20210118_01of01.xml


/content/drive/MyDrive/Python Engineer Assesment/DLTINS_20210118_01of01.xml


/content/drive/MyDrive/Python Engineer Assesment/DLTINS_20210119_01of02.xml


/content/drive/MyDrive/Python Engineer Assesment/DLTINS_20210119_02of02.xml


#Check Csv file

In [137]:
df=pd.read_csv('/content/drive/MyDrive/Python Engineer Assesment/DLTINS_20210119_01of02.csv')

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23316 entries, 0 to 23315
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   FinInstrmGnlAttrbts.Id              23316 non-null  object
 1   FinInstrmGnlAttrbts.FullNm          23316 non-null  object
 2   FinInstrmGnlAttrbts.ClssfctnTp      23316 non-null  object
 3   FinInstrmGnlAttrbts.CmmdtyDerivInd  23316 non-null  bool  
 4   FinInstrmGnlAttrbts.NtnlCcy         23316 non-null  object
 5   Issr                                23316 non-null  bool  
dtypes: bool(2), object(4)
memory usage: 774.3+ KB


In [139]:
df.head(2)

,FinInstrmGnlAttrbts.Id,FinInstrmGnlAttrbts.FullNm,FinInstrmGnlAttrbts.ClssfctnTp,FinInstrmGnlAttrbts.CmmdtyDerivInd,FinInstrmGnlAttrbts.NtnlCcy,Issr
0,DE000MA40TG3,Open End Turbo Long Fresenius emittiert von Mo...,RFSTCA,False,EUR,False
1,DE000MA40TH1,Open End Turbo Long Fresenius emittiert von Mo...,RFSTCA,False,EUR,False
